# 서울시 소방서 시각화

In [1]:
# api로 보내서 지도에 표시(못받아온 인덱스 확인 후 drop)
# 페이지 바꾸는 것은 셀레늄으로 바꾸고 BS로 크롤링

In [4]:
import numpy as np
import pandas as pd
import requests
from urllib.parse import quote
import folium, json

In [37]:
df = pd.read_csv('./csv/Seoul_Fire_Station.csv')
df.head()

,관서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119


In [38]:
df.tail()

,관서명,주소,전화번호
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119
99,송파소방서-마천-119 안전센터,서울특별시 송파구 오금로51길 56 (마천동),02-3401-2119


In [39]:
with open('../API/카카오API.txt') as file:
    kakao_key = file.read()

In [40]:
lat_list = []
lng_list = []
for i in df.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df.주소[i])}'
    result = requests.get(url,
      headers = {"Authorization":f'KakaoAK {kakao_key}'}).json()
    
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.관서명[i])

은평소방서-수색-119 안전센터


In [41]:
df[df.관서명.isin(["은평소방서-수색-119 안전센터"])].주소

30    서울특별시 은평구 수색로 294 (수색동)
Name: 주소, dtype: object

In [42]:
len(df)

100

In [43]:
df.drop(30, axis=0, inplace=True)

In [44]:
lat_list = []
lng_list = []
for i in df.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df.주소[i])}'
    result = requests.get(url,
      headers = {"Authorization":f'KakaoAK {kakao_key}'}).json()
    
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.관서명[i])

In [45]:
len(df)

99

In [46]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head()

,관서명,주소,전화번호,위도,경도
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190,37.494672,126.917719
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190,37.573205,126.935996
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119,37.544826,127.082779
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119,37.499833,127.142592
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119,37.530236,126.872342


In [47]:
df.tail()

,관서명,주소,전화번호,위도,경도
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380,37.516200,127.075940
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119,37.507316,127.093893
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059,37.519785,127.136572
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119,37.494163,127.151828
99,송파소방서-마천-119 안전센터,서울특별시 송파구 오금로51길 56 (마천동),02-3401-2119,37.499833,127.142592


In [50]:
# 만약 추가하고 싶다면
url = f'{search_url}?query={quote("서울특별시 은평구 수색로 320")}'
result = requests.get(url,
                     headers = {"Authorization":f'KakaoAK {kakao_key}'}).json()
print(result['documents'][0]['x'])
print(result['documents'][0]['y'])

126.889905679045
37.5852677029786


# 지도 시각화

In [51]:
map = folium.Map(location=[37.55, 126.98], zoom_start=12)

In [56]:
for i in df.index:
    folium.Marker(
        location = [df.위도[i], df.경도[i]],
        popup = folium.Popup(df.주소[i], max_width=200),
        tooltip = f'{df.관서명[i]}',
        icon = folium.Icon(icon = 'glyphicon glyphicon-home')
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울소재 소방서 위치</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map